# Explore langChain agent in handling numeric estimation tasks

## Goal
Use langChain agent to output reliable result.

## Task
we are tackling the task of mathematical computation.
-> already failed - rounding error
-> use step function? -> not useful
-> conclusion: langChain agent isn't easy to customised. The prompt wrapper is likely to contaminate the desired behaviour, and therefore it is extremly difficult to dictate model behavior.

## repot
Will not mention, but I can introduce this discovery in viva.

Benchmark
1. test suite designed with hardcoded python logic that embed randomly generated number

In [ ]:
%pip install openai
%pip install langchain
%pip install python-dotenv
%pip install numexpr

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

# Chain

A chain is made up from 'links'.

Types:
1. LLMChain = PromptTemplate + LLM
2. Agent - handle user input, pick tools/toolkits, and respond
3. Tool - functions that the agent use to execute / take actions
4. Toolkit - a set of tools for certain tasks

In [3]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
## set up LLM
llm = OpenAI(temperature=0.9)

## set up prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

## create a chain (or, chain the action)
chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run("beautiful socks").strip()

# Agent
Agent will dynamically call chains based on user input. HOW?

It will use LLM to determine which actions to take, and in what order.

Actions include:
1. use a tool and 'observe' the output
2. return to user

Documents: 
- Agents: [supported agents and their specifications](https://python.langchain.com/en/latest/modules/agents/getting_started.html)
- Tools: [predefined tools and their specifications](https://python.langchain.com/en/latest/modules/agents/tools/getting_started.html).

In [4]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

# First, let's load the language model we're going to use to control the agent. ()
gpt3 = OpenAI(temperature=0,model_name="text-davinci-003")

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["llm-math"], llm=gpt3)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, gpt3, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)



In [7]:
# Now let's test it out!
my_question = """**Assumptions**: Assume the Earth's ocean has an average depth of 3.7km and covers 71% of the Earth's surface. Assume the average density of fish in the ocean is 1 fish per 10 cubic meters of water.
**Calculations**: Calculate the volume of the ocean by multiplying the surface area of the Earth by the average depth of the ocean. Then, divide the volume of water by 10 to get the estimated number of fish. 
Write an executable python program to find the result of t.
"""
my_question = '''Sure! Let's apply the Fermi estimation technique and break down the problem of estimating the number of barber shops in London into smaller, more manageable subproblems.

1. Estimate the population of London: As of my knowledge cutoff in 2021, the population of London was around 9 million people.

2. Estimate the number of households in London: We can assume an average of 2.5 people per household, similar to the previous example. This would give us approximately 3.6 million households (9 million / 2.5).

3. Estimate the proportion of the population that needs barber services: We can assume that both men and women visit barber shops, but men might visit more frequently. Let's assume that 60% of the population needs barber services regularly.

4. Estimate the frequency of barber shop visits: Let's assume that the average person who visits a barber shop does so once every 6 weeks, or roughly 8.7 times a year (52 weeks / 6).

5. Estimate the number of customers a barber shop can serve in a day: Let's assume that a barber shop has 3 barbers, each of whom can serve 10 customers per day. This means a barber shop can serve 30 customers per day.

6. Estimate the number of days a barber shop is open per year: Let's assume that a barber shop is open 6 days a week for 50 weeks a year, which is 300 days.

Now, we can use these estimates to calculate the number of barber shops in London:

1. Calculate the number of people who need barber services: 9 million * 0.6 = 5.4 million people
2. Calculate the total number of barber visits per year: 5.4 million * 8.7 = 46.98 million visits
3. Calculate the total number of customers a barber shop can serve per year: 30 customers/day * 300 days = 9,000 customers/year
4. Estimate the number of barber shops needed to serve the population: 46.98 million visits / 9,000 customers/year = 5,220 barber shops

So, using Fermi estimation and breaking down the problem recursively, we can estimate that there are approximately 5,220 barber shops in London. Keep in mind that this is a rough estimation and the actual number may vary.'''
with get_openai_callback() as cb:
    output = agent.run(my_question)
    print(cb)
print(output)

# gpt3 will use natural log instead of proper log10
# the right prompt will be: "What is 32000 multiplied by 650 in the order of magnitude of 10?"



> Entering new AgentExecutor chain...
 I need to break down the problem into smaller, more manageable subproblems.
Action: None
Thought: I need to calculate the number of people who need barber services, the total number of barber visits per year, the total number of customers a barber shop can serve per year, and the number of barber shops needed to serve the population.
Action: Calculator
Action Input: 9 million * 0.6, 5.4 million * 8.7, 30 customers/day * 300 days, 46.98 million visits / 9,000 customers/year
Observation: None
Thought: I need to calculate the number of people who need barber services, the total number of barber visits per year, the total number of customers a barber shop can serve per year, and the number of barber shops needed to serve the population.
Action: Calculator is not a valid tool, try another one.
Thought: I need to calculate the number of people who need barber services, the total number of barber visits per year, the total number of customers a barber 

In [47]:
# initialize a chatGPT agent
# First, let's load the language model we're going to use to control the agent. ()
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
gpt3_5 = ChatOpenAI(temperature=0.9,model_name="gpt-3.5-turbo")
# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools([], llm=gpt3_5)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent_035 = initialize_agent(tools, gpt3_5, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

## set up prompt template
prompt = PromptTemplate(
    input_variables=["sentence"],
    template="""\
Assistant will repeat the sentence the user provides.\
Assistant will not engage in a conversation. Assistant will only repeat the sentence.\
SENTENCE: what is 32000 multiplied by 650 in the order of magnitude of 10?
OUTPUT: what is 32000 multiplied by 650 in the order of magnitude of 10?

SENTENCE: is there a way to get the result of t?
OUTPUT: is there a way to get the result of t?

SENTENCE: I don't want to do anything.
OUTPUT: I don't want to do anything.

SENTENCE: I want to do something.
OUTPUT: I want to do something.

SENTENCE: {sentence}
QUESTION: """,
)

sentence = "Assume the distance from Boston, MA to Portugal is 5,000 km. Assume the width of the bridge is 1 meter and the height is 0.5 meters. Assume the average thickness of a penny is 1.5 mm."
with get_openai_callback() as cb:
    output = agent_035.run(prompt)
    print(cb)

# CLAIM: change temperature from 0 to 0.9 can revolve the math error - because it will stop using rounding expression!!

# if I use this prompt: "What is 32000 multiplied by 650 in the order of magnitude of 10?"
# the second calculator action will use log instead of log10

# if I use this prompt: "What is 32000 multiplied by 650 in the order of magnitude of 10? Use log10"
# the first calculator action will use log10(32000*650) and second calculator action will result in error.

# conclusion: avoid letting the model generate expression like round()



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `The template is just asking the assistant to repeat back the user's sentence, so no specific action is required.`

In [17]:
gpt4 = ChatOpenAI(temperature=0.5,model_name="gpt-4")
agent_004 = initialize_agent(tools, gpt4, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
with get_openai_callback() as cb:
    # output = agent_004.run("What is 32000 multiplied by 650 in the order of magnitude?")
    output = agent_004.run(f"Round this number to an integer: {7.318063349}")
    print(cb)




> Entering new AgentExecutor chain...
I need to round the number to the nearest integer.
Action: Calculator
Action Input: round(7.318063349)

ValueError: LLMMathChain._evaluate("
round(7.318063349)
") raised error: 'VariableNode' object is not callable. Please try again with a valid numerical expression

## Useful tools for Fermi-aid:

**python_repl** - a python shell
**wolfram-alpha** - need app id to query wolfram-alpha
**pal-math** - a LLM solving fully worded hard word math problem
**llm-math** - a LLMMath chain instance
**wikipedia** - use wikipedia python package to call MediaWiki API and parse results

Other: ChatGPT Plugin, Hugging Face


## Create a customised chain

In [29]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain
from langchain.prompts import PromptTemplate
from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [33]:
prompt_1 = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1,verbose=True)

prompt_2 = PromptTemplate(
    input_variables=["product"],
    template="What is a good slogan for a company that makes {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2,verbose=True)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2, verbose=True)
concat_output = concat_chain.run("colorful socks")
print(concat_output)



> Entering new ConcatenateChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
What is a good name for a company that makes colorful socks?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
What is a good slogan for a company that makes colorful socks?

> Finished chain.

> Finished chain.


Socktastic!

"Step Into Colorful Comfort!"
